# Use case 2

Task 1

In [ ]:
# Code for annovar
# Generate protein sequences with annovar
cd /mnt/e/360Downloads/Software/WSL/annovar
tar -xvzf annovar.latest.tar.gz
ls –ltr
ls ./annovar/
vim test.vcf

./annovar/convert2annovar.pl -format vcf4 test.vcf > test_annovar.avinput  # given in slide
ls -ltr
less test_annovar.avinput  # View the file content

./annovar/table_annovar.pl test_annovar.avinput ./annovar/humandb/ -buildver hg19 -out test_annovar -protocol refGene -operation g -polish # given in slide
ls -ltr
less test_annovar.refGene.exonic_variant_function

./annovar/coding_change.pl test_annovar.refGene.exonic_variant_function ./annovar/humandb/hg19_refGene.txt ./annovar/humandb/hg19_refGeneMrna.fa -includesnp -onlyAltering > test_annovar.mutated_protein.fasta
ls -ltr
less test_annovar.mutated_protein.fasta

In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 78.9 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO
import re

In [ ]:
input_fasta_file = "/content/test_annovar.mutated_protein.fasta"
output_file = "/content/extracted_peptides.txt"

mutation_aa = "L"  # Mutated amino acid, recommended to be detected automatically from the FASTA file
fixed_length = 15  # Fixed-length peptide
min_length = 12  # Minimum peptide length
max_length = 25  # Maximum peptide length

# Function to automatically detect the mutated amino acid
def detect_mutation_aa(wild_seq, mutant_seq):
    for i, (w, m) in enumerate(zip(wild_seq, mutant_seq)):
        if w != m:
            return m
    return None

# Function to extract peptides (fixed-length and variable-length)
def extract_fixed_peptides(sequence, mutation_aa, fixed_length):
    peptides = []
    for i in range(len(sequence) - fixed_length + 1):
        if sequence[i] == mutation_aa and sequence[i + fixed_length - 1] == mutation_aa:
            peptides.append(sequence[i:i + fixed_length])
    return peptides

def extract_variable_peptides(sequence, mutation_aa, min_length, max_length):
    peptides = []
    for i in range(len(sequence)):
        if sequence[i] == mutation_aa:
            for j in range(i + min_length, min(len(sequence), i + max_length) + 1):
                if sequence[j - 1] == mutation_aa:
                    peptides.append(sequence[i:j])
    return peptides

with open(output_file, "w") as out_file:
    for record in SeqIO.parse(input_fasta_file, "fasta"):
        sequence = str(record.seq)

        # Extract fixed-length peptides
        fixed_peptides = extract_fixed_peptides(sequence, mutation_aa, fixed_length)

        # Extract variable-length peptides (excluding fixed-length)
        variable_peptides = extract_variable_peptides(sequence, mutation_aa, min_length, max_length)
        variable_peptides = [pep for pep in variable_peptides if len(pep) != fixed_length]

        # Write results to the output file
        out_file.write(f">{record.id}_fixed_length_{fixed_length}\n")
        for peptide in fixed_peptides:
            out_file.write(peptide + "\n")

        out_file.write(f">{record.id}_variable_length_{min_length}-{max_length}\n")
        for peptide in variable_peptides:
            out_file.write(peptide + "\n")

print(f"Extraction completed, results saved to {output_file}")

提取完成，结果已保存至 /content/extracted_peptides.txt


In [ ]:
fasta_file = "/content/test_annovar.mutated_protein.fasta"

# Compare two sequences to identify mutation positions and details
def detect_mutations(wild_seq, mutant_seq):
    mutations = []
    for i, (w, m) in enumerate(zip(wild_seq, mutant_seq)):
        if w != m:
            mutations.append((i + 1, w, m))  # Position starts from 1
    return mutations

sequences = list(SeqIO.parse(fasta_file, "fasta"))
if len(sequences) >= 2:
    wild_type_seq = str(sequences[0].seq)  # The first sequence is wild-type
    mutant_seq = str(sequences[1].seq)    # The second sequence is mutant-type

    mutations = detect_mutations(wild_type_seq, mutant_seq)

    print("Mutation positions:")
    for pos, wild, mutant in mutations:
        print(f"Position {pos}: {wild} -> {mutant}")
else:
    print("The FASTA file does not contain enough sequences for comparison (at least wild-type and mutant-type are needed).")

Mutation positions:
Position 10: G -> D


In [ ]:
fasta_file = "/content/test_annovar.mutated_protein.fasta"

# Extract the first gene
with open(fasta_file, "r") as file:
    records = list(SeqIO.parse(file, "fasta"))  # Parse the FASTA file
    if len(records) > 0:
        first_record = records[0]  # Get the first gene record
        gene_id = first_record.id  # Gene ID
        gene_description = first_record.description  # Gene description
        gene_sequence = str(first_record.seq)  # Gene sequence

        print("Information of the first gene:")
        print(f"Gene ID: {gene_id}")
        print(f"Description: {gene_description}")
        print(f"Sequence: {gene_sequence}")
    else:
        print("No gene information found in the FASTA file!")

第一个基因的信息：
基因ID: line1
描述: line1 NM_016255 WILDTYPE
序列: MAEGPEEARGHPPGQDDGGGDHEPVPSLRGPPTTAVPCPRDDPQAEPQAPGRPTAPGLAAAAAADKLEPPRELRKRGEAASGSGAELQEQAGCEAPEAAAPRERPARLSAREYSRQVHEWLWQSYCGYLTWHSGLAAFPAYCSPQPSPQSFPSGGAAVPQAAAPPPPQLGYYNPFYFLSPGAAGPDPRTAAGISTPAPVAGLGPRAPHVQASVRATPVTRVGSAAPSRSPSETGRQAGREYVIPSLAHRFMAEMVDFFILFFIKATIVLSIMHLSGIKDISKFAMHYIIEEIDEDTSMEDLQKMMVVALIYRLLVCFYEIICIWGAGGATPGKFLLGLRVVTCDTSVLIAPSRVLVIPSSNVSITTSTIRALIKNFSIASFFPAFITLLFFQHNRTAYDIVAGTIVVKRNGVR*


In [ ]:
fasta_file = "/content/test_annovar.mutated_protein.fasta"
output_file = "extracted_peptides_length_20_final.txt"

# Fixed peptide length
fixed_length = 20

def parse_mutation(header):
    """
    Parse mutation information, including point mutations and frameshift mutations.
    If parsing fails, return (None, None, None).
    """
    # Detect frameshift mutations
    fs_match = re.search(r"p\.([A-Z])(\d+)([A-Z]*)fs\*?\d*", header)
    if fs_match:
        position = int(fs_match.group(2))  # Mutation position
        mutated_aa = fs_match.group(3) or ""
        return position, mutated_aa, "frameshift"

    # Detect normal mutations
    normal_match = re.search(r"p\.([A-Z])(\d+)([A-Z\*])", header)
    if normal_match:
        position = int(normal_match.group(2))
        mutated_aa = normal_match.group(3)
        return position, mutated_aa, "normal"

    return None, None, None

def extract_peptides(sequence, mutation_position, fixed_length):
    """
    Extract all 15-mer peptides containing the mutation site from the mutated protein sequence.
    The mutation site shifts from the last position to the first position within the peptide, attempting 15 different offsets.
    """
    peptides = []
    # Since mutation_position is 1-based, convert it to 0-based
    start_idx = mutation_position - 1

    for offset in range(fixed_length):
        start = start_idx - (fixed_length - 1) + offset
        end = start + fixed_length

        if start >= 0 and end <= len(sequence):
            peptide = sequence[start:end]
            if "*" in peptide:
                continue
            peptides.append(peptide)

    return peptides

with open(output_file, "w") as out_file:
    for record in SeqIO.parse(fasta_file, "fasta"):
        header = record.description
        seq = str(record.seq)

        # Parse mutation information
        mut_pos, mut_aa, mut_type = parse_mutation(header)

        # If no mutation is detected, output a warning
        if mut_pos is None:
            out_file.write(f">{header}\nNo valid mutation detected.\n")
            continue

        # Remove all '*' characters from the sequence for both frameshift and normal mutations
        seq = seq.replace("*", "")

        # Extract 15-mer peptides containing the mutation site
        peptides_15mer = extract_peptides(seq, mut_pos, fixed_length)

        out_file.write(f">{header}\n")
        out_file.write(f"Mutation Type: {mut_type}\n")
        if peptides_15mer:
            for p in peptides_15mer:
                out_file.write(p + "\n")
        else:
            out_file.write("No valid 15-mer peptides found.\n")

print(f"Peptide extraction complete, results saved to {output_file}")

肽段提取完成，结果保存至 extracted_peptides_length_20_final.txt
